In [304]:
from dash import dcc, html, Dash, Input, Output, State, callback
import pandas as pd, wget, plotly.express as px

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [305]:
df.sample(5)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
46,46,43,KSC LC-39A,1,5200.00,F9 FT B1031.2,FT
2,2,3,CCAFS LC-40,0,525.00,F9 v1.0 B0005,v1.0
39,39,33,KSC LC-39A,1,3696.65,F9 FT B1032.1,FT
44,44,39,KSC LC-39A,1,3310.00,F9 B4 B1039.1,B4
4,4,5,CCAFS LC-40,0,677.00,F9 v1.0 B0007,v1.0


In [306]:
filtered_df = df[df["class"] == 1]
filtered_df
df.groupby("Launch Site")["class"].count().reset_index()

,Launch Site,class
0,CCAFS LC-40,26
1,CCAFS SLC-40,7
2,KSC LC-39A,13
3,VAFB SLC-4E,10


In [307]:
# Group by "Launch Site" and "class", then count the occurrences
grouped_df = df.groupby(["Launch Site", "class"]).size().unstack()

grouped_df.columns = ["unsuccessful", "successful"]

grouped_df



,unsuccessful,successful
Launch Site,,
CCAFS LC-40,19,7
CCAFS SLC-40,4,3
KSC LC-39A,3,10
VAFB SLC-4E,6,4


In [331]:
import numpy as np
app = Dash(__name__)

app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={"color" : "brown", "text-align" : "center"}),
    html.Div([


        # DROPDOWN SECTION
          dcc.Dropdown(id='site-dropdown',
                       
                options=[
                    {'label': launch_site , 'value': launch_site}
                    for launch_site in np.insert(df["Launch Site"].unique(), 0, "ALL")
                ],
                value='ALL',
                placeholder="place holder here",
                searchable=True
                ),

            html.Div([
                dcc.Graph(id="success-pie-chart")
            ]),



        #SLIDER SECTION

            html.H2("Payload Mass (Kg)"),
            dcc.RangeSlider(
                id='payload-slider',
                min=0, max=10000, step=1000,
                value=[0, 10000],
                marks={
                        0: '0',
                       2500 : "2500",
                       5000 : "5000",
                       7500 : "7500",
                       10000: '10000'},
                ),




        html.Div([
            
            dcc.Graph(id="success-payload-scatter-chart")
        ])
            
    ])
])




@app.callback(
            [Output(component_id='success-pie-chart', component_property='figure'),
             Output(component_id='success-payload-scatter-chart', component_property='figure')],

            [Input(component_id='site-dropdown', component_property='value'), 
            Input(component_id="payload-slider", component_property="value")]
)


def get_pie_chart(entered_site, entered_range):
    filtered_df = df[
        (df["Payload Mass (kg)"] >= entered_range[0]) & 
        (df["Payload Mass (kg)"] <= entered_range[1])
    ]

    if entered_site == 'ALL':
        pie_fig = px.pie(filtered_df[filtered_df["class"] == 1],
                         names='Launch Site',
                         title='Total Success Launches By Site')
    else:
        filtered_df = filtered_df[filtered_df["Launch Site"] == entered_site]
        pie_fig = px.pie(filtered_df,
                         names='class',
                         title=f'Success vs Unsuccess for site {entered_site}')

    scatter_fig = px.scatter(filtered_df,
                             x="Payload Mass (kg)", y="class", 
                             color="Booster Version Category",
                             title=f'Correlation Between Payload and Success for {entered_site if entered_site != "ALL" else "All Sites"}')

    return pie_fig, scatter_fig



if __name__ == '__main__':
    app.run(host="127.0.0.1", port="8051")